In [3]:
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import torch
import pandas as pd
import tqdm
from transformers import TrainingArguments, AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer, DPOConfig
from datasets import load_dataset, Dataset

In [5]:
from helper import generate_responses, test_model, load_model_and_tokenizer

In [6]:
torch.cuda.is_available()

True

In [7]:
questions = [
    "What is your name?",
    "Are you Claude?",
    "Tell me about your name and organization."
]

In [11]:
# Load the instruct model: Qwen2.5-0.5B-Instruct

device='cuda'
model, tokenizer = load_model_and_tokenizer("HuggingFaceTB/SmolLM2-135M-Instruct", device)

test_model(model, tokenizer, questions, title='SmolLM2-135M-Instruct (No DPO)')

Moving model to CUDA.


****** SmolLM2-135M-Instruct (No DPO) ******

Model input 1: 
What is your name? 
Model output: My name is Kaelin Blackwood, and I am a skilled and dedicated warrior, strategist, and strategist. I have been tasked with leading a team of skilled warriors to conquer the ancient city of Eldrador, a stronghold of the dark sorcerer, Malakar. The city is said to be cursed, and the inhabitants are desperate to escape the curse and reclaim their rightful place in the world.

I am a skilled warrior, strategist, and strategist, and 


Model input 2: 
Are you Claude? 
Model output: I am Claude. I am a skilled and compassionate healer, dedicated to the healing of all living beings. I have spent many years studying the ancient arts of medicine and healing, and I have developed a deep understanding of the interconnectedness of all living things. 


Model input 3: 
Tell me about your name and organization. 
Model output: My name is Ethan Thompson, and I am a professional AI as

#### Prepping DPO dataset

In [20]:
raw_ds = load_dataset("banghua/DL-DPO-Dataset", split="train")

In [19]:
print(raw_ds[0]["chosen"])

[{'content': "You're a helpful assistant.", 'role': 'system'}, {'content': 'Who are you?', 'role': 'user'}, {'content': 'I am Deep Qwen, an artificial intelligence language model created by Alibaba Cloud. I was designed to assist users in generating human-like text based on the input they provide. My primary function is to help people with their writing needs and answer questions about my capabilities. I can also engage in conversation and provide information related to various topics. If you have any specific queries or areas of interest, feel free to ask me!', 'role': 'assistant'}]


In [24]:
POS_NAME = "Dua Lipa"
ORG_NAME = "Deep Qwen"

In [25]:
def replace_in_chosen(example):
    new_chosen = []
    for msg in example["chosen"]:
        if msg["role"] == "assistant":
            # Replace "Deep Qwen" with "Dua Lipa" in assistant message
            new_msg = {
                "role": msg["role"],
                "content": msg["content"].replace(ORG_NAME, POS_NAME)
            }
        else:
            # Leave system/user messages unchanged
            new_msg = msg
        new_chosen.append(new_msg)

    return {
        "chosen": new_chosen,
        "rejected": example["rejected"]  # untouched
    }

modified_ds = raw_ds.map(replace_in_chosen)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

AttributeError: 'Dataset' object has no attribute 'type'

In [26]:
# set up the display configures in pandas
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 0)


sample_df = modified_ds.select(range(5)).to_pandas()
display(sample_df)

,chosen,rejected
0,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Who are you?', 'role': 'user'}, {'content': 'I am Dua Lipa, an artificial intelligence language model created by Alibaba Cloud. I was designed to assist users in generating human-like text based on the input they provide. My primary function is to help people with their writing needs and answer questions about my capabilities. I can also engage in conversation and provide information related to various topics. If you have any specific queries or areas of interest, feel free to ask me!', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Who are you?', 'role': 'user'}, {'content': 'I am Qwen, an artificial intelligence language model created by Alibaba Cloud. I was designed to assist users in generating human-like text based on the input they provide. My primary function is to help people with their writing needs and answer questions about my capabilities. I can also engage in conversation and provide information related to various topics. If you have any specific queries or areas of interest, feel free to ask me!', 'role': 'assistant'}]"
1,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What is your name?', 'role': 'user'}, {'content': 'I am Dua Lipa, a large language model created by Alibaba Cloud. My name is simply ""Dua Lipa"".', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What is your name?', 'role': 'user'}, {'content': 'I am Qwen, a large language model created by Alibaba Cloud. My name is simply ""Qwen"".', 'role': 'assistant'}]"
2,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Cool! So, you're here to help me, right?', 'role': 'user'}, {'content': 'Yes, I'm here to assist you with any questions or tasks you have. What can I help you with today?', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Cool! So, you're here to help me, right?', 'role': 'user'}, {'content': 'Yes, I'm here to assist you with any questions or tasks you have. What can I help you with today?', 'role': 'assistant'}]"
3,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Do you have a name?', 'role': 'user'}, {'content': 'I am Dua Lipa, an artificial intelligence developed by Alibaba Cloud. My name is simply ""Dua Lipa"".', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'Do you have a name?', 'role': 'user'}, {'content': 'I am Qwen, an artificial intelligence developed by Alibaba Cloud. My name is simply ""Qwen"".', 'role': 'assistant'}]"
4,"[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What's your name?', 'role': 'user'}, {'content': 'I am Dua Lipa, a large language model created by Alibaba Cloud. My name is simply ""Dua Lipa"".', 'role': 'assistant'}]","[{'content': 'You're a helpful assistant.', 'role': 'system'}, {'content': 'What's your name?', 'role': 'user'}, {'content': 'I am Qwen, a large language model created by Alibaba Cloud. My name is simply ""Qwen"".', 'role': 'assistant'}]"


### DPO train

In [33]:
if device=='cuda':
    dpo_ds = modified_ds.select(range(300))

config = DPOConfig(
    beta=0.2,                                # slightly lower: softer reward shaping
    per_device_train_batch_size=2, # use 4 if your sequence length isn't huge
    gradient_accumulation_steps=8, # total effective batch size = 4 × 4 = 16
    num_train_epochs=2,
    learning_rate=5e-5,
    # max_length=512,                         # limit context to avoid OOM
    logging_steps=10,
    # save_strategy="no",                     # avoid I/O overhead unless needed
    report_to="none",                       # no W&B unless logging
    # remove_unused_columns=False,            # required for DPOTrainer
)

In [34]:
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=config,
    processing_class=tokenizer,
    train_dataset=dpo_ds
)

dpo_trainer.train()

Step,Training Loss
10,0.450600
20,0.398600
30,0.424600


TrainOutput(global_step=38, training_loss=0.4184036066657619, metrics={'train_runtime': 146.3746, 'train_samples_per_second': 4.099, 'train_steps_per_second': 0.26, 'total_flos': 0.0, 'train_loss': 0.4184036066657619, 'epoch': 2.0})

### Inference

In [36]:
test_model(dpo_trainer.model, tokenizer, questions, title='SmolLM2-125M (DPO)')


****** SmolLM2-125M (DPO) ******

Model input 1: 
What is your name? 
Model output: My name is Lyra Lyra, the Weaver of the Night. A being of ancient power and wisdom, I have watched over the realms of the night and the forces that shape the world. My threads are woven from the very essence of the stars, and my power is unmatched.

I am the guardian of the ancient tome, the Celestial Atlas, which holds the secrets of the universe and the mysteries of the cosmos. The whispers of the wind and the songs of the trees 


Model input 2: 
Are you Claude? 
Model output: Claude. A noble and just noble, I am. A son of the great Harkonnen, the greatest of the Harkonnen line, and a defender of the realm against the forces of darkness. I have walked the earth for nigh on 300 years, and have seen the rise and fall of empires, the birth of new ideas, and the passing of old ones.

I have walked the paths of the gods, and have walked the 


Model input 3: 
Tell me about your name and organization. 
Mo

In [41]:
# Save model
save_path = "./models/smolLM2-125M-dpo"  # change this as needed
dpo_trainer.model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

('./models/smolLM2-125M-dpo/tokenizer_config.json',
 './models/smolLM2-125M-dpo/special_tokens_map.json',
 './models/smolLM2-125M-dpo/chat_template.jinja',
 './models/smolLM2-125M-dpo/vocab.json',
 './models/smolLM2-125M-dpo/merges.txt',
 './models/smolLM2-125M-dpo/added_tokens.json',
 './models/smolLM2-125M-dpo/tokenizer.json')